# prop_info data model

**Lucas M. Hale**, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), *Materials Science and Engineering Division, NIST*.

**Chandler A. Becker**, [chandler.becker@nist.gov](mailto:chandler.becker@nist.gov?Subject=ipr-demo), *Office of Data and Informatics, NIST*.

**Zachary T. Trautt**, [zachary.trautt@nist.gov](mailto:zachary.trautt@nist.gov?Subject=ipr-demo), *Materials Measurement Science Division, NIST*.

Version: 2016-09-02

[Disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm) 
 
- - -

## Introduction

The prop_info data model used by [atom_dump.load](../reference/atomman.lammps.atom_dump.load.ipynb) and [atom_dump.dump](../reference/atomman.lammps.atom_dump.dump.ipynb) gives metadata information to LAMMPS dump files allowing for proper unit control and conversions between dump files and [atomman.System](../reference/atomman.System.ipynb) objects.

- - -

## Description

The prop_info data model is consistent with a json/xml structure. The root element of the model is __"LAMMPS-dump-atoms_prop-relate"__.

Elements:

- __"box_prop"__ -- contains metadata associated with the box vectors

- __"atoms_prop"__ -- gives information for defining per-atom properties in atomman

- __"LAMMPS-attribute"__ -- defines the relationships between atomman's properties and the dump file's attributes.

### The box_prop branch

The __"box_prop"__ branch gives metadata associated with the box vectors. Right now it is only used to define what units the LAMMPS box vectors are in.

Subelements:

- __"unit"__ -- defines the units associated with the box vectors in the LAMMPS dump file. If not specified or the value is null then no conversion is performed.

Usage:

- [atom_dump.load](../reference/atomman.lammps.atom_dump.load.ipynb) -- when the box vectors are read in from the dump file they are converted from __"unit"__ to atomman's working length unit (Angstroms by default).

- [atom_dump.dump](../reference/atomman.lammps.atom_dump.dump.ipynb) -- when the box vectors are written out to the dump file they converted from atomman's working length unit (Angstroms by default) to __"unit"__.

In [ ]:
"""
{
    "box_prop": {
        "unit": <length-unit>
    }
}
"""

### The atoms_prop branch

The __"atoms_prop"__ branch of the data model allows for the names, shapes and data types of the various atomic properties as stored in the atomman System.

Subelements:

- The names of the subelements of __"atoms_prop"__ correspond to the per-atom properties as represented in an [atomman.System](../reference/atomman.System.ipynb). 

    - __"dtype"__ -- is a string specifying the data type for the property ("int", "float", or "bool").  If this term is not listed or has a null value then the data type is inferred from the values.

    - __"shape"__ -- specifies the property's data shape. If this term is not listed or is null, then the property is made scalar. An integer creates a vector property of the specified length. A list of integers creates a tensor with the specified dimensions and shape.  
    
Usage:

- [atom_dump.load](../reference/atomman.lammps.atom_dump.load.ipynb) -- all the specified properties in __"atoms_prop"__ are created with initial values of zero.

- [atom_dump.dump](../reference/atomman.lammps.atom_dump.dump.ipynb) -- not used.

In [ ]:
"""
{
    "atoms_prop": {
        <prop-name>: {
            "dtype": <data-type>,
            "shape": <shape-value>
        },
        <prop-name>: {
            "dtype": <data-type>,
            "shape": <shape-value>
        }
    }
}
"""

### The LAMMPS-attribute branch

The __"LAMMPS-attribute"__ branch of the data model defines which attributes (atomic properties) to read/write in the LAMMPS dump file and how to interpret and convert them to properties in the [atomman.System](../reference/atomman.System.ipynb). 

Subelements:

- The names of the subelements of __"LAMMPS-attribute"__ correspond to the names of the attributes in the LAMMPS dump file to read/write. 

    - __"relation"__ -- provides conversion information for relating an attribute in the dump file to an atomic property in the System. Each attribute can have multiple __"relation"__ elements (i.e. can be a list).   

        - __"prop"__ -- gives the name of the System's property name to relate the attribute to. This has to be specified for each relation.

        - __"index"__ -- gives the index value associated with __"prop__" to relate the attribute to. All dump attributes are per-atom scalars, so providing an __"index"__ is necessary in relating the attribute to components of vector and tensor properties of the System.

        - __"unit"__ -- defines the units associated with the attribute in the LAMMPS dump file. A value of "scaled" indicates that it is a component of a 3D vector scaled relative to the box vectors. If __"unit"__ is not specified or the value is null then no conversion is performed.
   
Usage:

- [atom_dump.load](../reference/atomman.lammps.atom_dump.load.ipynb) -- the specified attribute names are read in, values are converted from __"unit"__ to atomman's working units, and the values are saved to the property values specified by __"prop"__ (and __"index"__).

- [atom_dump.dump](../reference/atomman.lammps.atom_dump.dump.ipynb) -- an array of values corresponding to all attributes for all atoms is constructed. Property values specified by __"prop"__ (and __"index"__) are converted from atomman's working units to __"unit"__ and saved to the array, which is then exported to the dump file. In the case where multiple __"relation"__s are given for an attribute, only the first __"relation"__ is used (values from all relations should be numerically equivalent anyways). 

In [ ]:
"""
{
    "LAMMPS-attribute": {
        <attribute>: {
            "relation" : [
                {
                    "prop": <prop-name>,
                    "index": <index-value>,
                    "unit": <unit-value>
                },
                {
                    "prop": <prop-name>,
                    "index": <index-value>,
                    "unit": <unit-value>
                }
            ]
        },
        <attribute>: {
            "relation" : {
                "prop": <prop-name>,
                "index": <index-value>,
                "unit": <unit-value>
            }
        }
    }
}
"""

### Default prop_info

The easiest way to create an instance of the data model is to first call [atom_dump.load](../reference/atomman.lammps.atom_dump.load.ipynb) or [atom_dump.dump](../reference/atomman.lammps.atom_dump.dump.ipynb) by specifying __fname__ but not __prop_info__.  This will create and save a default data model to [__fname__].json. 

__Note:__ The default data models created by load() and dump() are different.  However, both default models allow for System-dump conversions of all the data forwards and backwards. 

#### Default prop_info from [atom_dump.load](../reference/atomman.lammps.atom_dump.load.ipynb)

An example of a default prop_info constructed by this function can be found in the [atom_dump.load Notebook](../reference/atomman.lammps.atom_dump.load.ipynb)

Default conversion rules:

1. Box units set to null (no conversion).

2. Property "atype" is defined as a scalar int and attribute "type" is related to it.

3. Property "pos" is defined as a 3D vector of floats. The attributes related to property "pos" depend on what xyz attributes are in the dump file. The search logic looks for a full set of ["x", "y", "z"], ["xs", "ys", "zs"], ["xu", "yu", "zu"], or ["xsu", "ysu", "zsu"] in this listed order. For the "-s" or "-su" sets the units of the attributes are specified as "scaled", otherwise units are not given.

4. All attributes in the dump file except "id", "type", and the xyz-set related to property "pos" are related to scalar properties with matching names. No units are given for the attributes. No data types are given for the properties.

#### Default prop_info from [atom_dump.dump](../reference/atomman.lammps.atom_dump.dump.ipynb)

An example of a default prop_info constructed by this function can be found in the [atom_dump.dump Notebook](../reference/atomman.lammps.atom_dump.dump.ipynb)

Default conversion rules:

1. Box units set to null (no conversion).

2. Property "atype" is defined as a scalar int and attribute "type" is related to it.

3. Property "pos" is defined as a 3D vector of floats and attributes "x", "y", and "z" are related to it. Units are not given for the attributes.

4. All properties except "atype" and "pos" are defined with their appropriate data types and shapes. An attribute is created for every component of every property. The names of the attributes match the property names, with attributes based on vector and tensor properties having bracketed index(es) at the ends of the names. Index values in the names start with 1 to coincide with how LAMMPS prints vector quantities to dump files. No units are given for the attributes.

- - -
__Docs Navigation:__

Tutorial:

1. [Basics](../tutorial/1 Basics.ipynb)

2. [LAMMPS Functionality](../tutorial/2 LAMMPS Functionality.ipynb)

3. [Defect Generation and Evaluation](../tutorial/3 Defect Generation and Evaluation.ipynb)


Reference:

- [atomman](../reference/atomman.ipynb)

- [atomman.convert](../reference/atomman.convert.ipynb)

- [atomman.defect](../reference/atomman.defect.ipynb)

- [atomman.lammps](../reference/atomman.lammps.ipynb)

- [atomman.tools](../reference/atomman.tools.ipynb)

- [atomman.unitconvert](../reference/atomman.unitconvert.ipynb)